In [ ]:
# Library

import torch
import math
import re
import numpy as np
import pandas as pd
from torch.distributions import Laplace
import ast, time
from tqdm import tqdm
import pandas as pd

from transformers import AutoTokenizer
from transformers import AutoModel
from transformers import AutoModelForCausalLM
from transformers import GPT2LMHeadModel

from datasets import load_dataset
from sentence_transformers import SentenceTransformer, util
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from scipy.stats import vonmises_fisher
import torch.nn.functional as F

from typing import Dict, List, Optional

from openai import OpenAI
import glob
import os
from collections import Counter

from sentence_transformers import SentenceTransformer
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
client = OpenAI(api_key="Your_API_Key")

In [ ]:
# --- Load tokenizer and GPT-2 model ---
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2", output_hidden_states=True)
embedding_table = model.get_input_embeddings().weight.detach()
if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token
model.eval()

# Base tokenizer pad fix (optional)
if getattr(tokenizer, "pad_token_id", None) is None and getattr(tokenizer, "eos_token", None) is not None:
    tokenizer.pad_token = tokenizer.eos_token

# Load a light GPT-2 model
gpt2_tok = AutoTokenizer.from_pretrained("distilgpt2")
if gpt2_tok.pad_token is None:
    gpt2_tok.pad_token = gpt2_tok.eos_token
gpt2_model = AutoModelForCausalLM.from_pretrained("distilgpt2").to(
    "cuda" if torch.cuda.is_available() else "cpu"
).eval()

# --- Extract embedding table ---
# Normalize embedding table for search
norm_embedding_table = torch.nn.functional.normalize(embedding_table, dim=1)

In [ ]:
# Get the answers

def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        return re.sub(r'[^\w\s]', '', text)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

def get_tokens(s):
    if not s: return []
    return normalize_answer(s).split()

def compute_f1(prediction, truth):
    pred_tokens = get_tokens(prediction)
    truth_tokens = get_tokens(truth)

    # if either is empty, return 1 if both are empty, 0 otherwise
    if not pred_tokens and not truth_tokens:
        return 1.0
    if not pred_tokens or not truth_tokens:
        return 0.0

    common_tokens = Counter(pred_tokens) & Counter(truth_tokens)
    num_common = sum(common_tokens.values())

    if num_common == 0:
        return 0.0

    precision = num_common / len(pred_tokens)
    recall = num_common / len(truth_tokens)

    return (2 * precision * recall) / (precision + recall)

def evaluate_answer(predicted_answer: str, true_answers: list[str]):
    """
    Evaluates a predicted answer against a list of true answers using F1 score and exact match.

    Args:
        predicted_answer: The generated answer string.
        true_answers: A list of true answer strings.

    Returns:
        A tuple containing:
            - The maximum F1 score achieved against any of the true answers.
            - A boolean indicating whether an exact match was found against any of the true answers.
    """
    max_f1 = 0.0
    exact_match = False

    normalized_prediction = normalize_answer(predicted_answer)

    for true_answer in true_answers:
        normalized_true = normalize_answer(true_answer)

        # Exact Match
        if normalized_prediction == normalized_true:
            exact_match = True

        # F1 Score
        f1 = compute_f1(predicted_answer, true_answer)
        max_f1 = max(max_f1, f1)

    return max_f1, exact_match


def calculate_cosine_similarity(prediction: str, truths: list[str], model) -> float:
    """
    Calculates the cosine similarity between the prediction and each true answer
    using Sentence-BERT embeddings and returns the maximum similarity.
    """
    if not prediction or not truths:
        return 0.0

    # Encode the prediction
    prediction_embedding = model.encode(prediction, convert_to_tensor=True)

    max_similarity = 0.0
    for truth in truths:
        if not truth:
            continue
        # Encode the true answer
        truth_embedding = model.encode(truth, convert_to_tensor=True)

        # Calculate cosine similarity
        similarity = util.pytorch_cos_sim(prediction_embedding, truth_embedding).item()
        max_similarity = max(max_similarity, similarity)

    return max_similarity


In [ ]:
# --- Load all INF Polar and INF Laplace datasets ---
csv_files_polar_inf = glob.glob("/content/squad_inf_polar_sweep_avg_epsilon_*.csv")
csv_files_laplace_inf = glob.glob("/content/squad_inf_laplace_sweep_avg_epsilon_*.csv")

csv_files = csv_files_polar_inf + csv_files_laplace_inf
print("Files found by glob:", csv_files)

dataframes = {}

for file in csv_files:
    df_name = file.replace(".csv", "")
    dataframes[df_name] = pd.read_csv(file)

print("Number of dataframes loaded:", len(dataframes))

In [ ]:
# Initialize Sentence Transformer model
model_sentence = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
num_rounds = 3
all_results = []

for round_num in range(num_rounds):
    print(f"--- Starting Round {round_num + 1}/{num_rounds} ---")
    results_list = []

    for df_name, df in dataframes.items():
        print(f"Processing DataFrame: {df_name}")
        for index, row in df.iterrows():
            question = row["question"]
            true_answers = eval(row["answers"]) # Assuming 'answers' is stored as a string representation of a list
            privatized_context = row["privatized_context"]
            repaired_context = row["repaired_context"]

            # Generate answer for privatized context
            try:
                response_privatized = client.chat.completions.create(
                    model="gpt-4o-mini",
                    messages=[
                        {"role": "system", "content": "You are a helpful assistant that answers questions based on the provided context. Limit your answer to one word."},
                        {"role": "user", "content": f"Context: {privatized_context}\nQuestion: {question}"}
                    ],
                    temperature=0.2,
                    max_tokens=5, # Limit response to one word, but just in case
                )
                answer_privatized = response_privatized.choices[0].message.content.strip()
            except Exception as e:
                print(f"Error generating answer for privatized context: {e}")
                answer_privatized = ""


            # Generate answer for repaired context
            try:
                response_repaired = client.chat.completions.create(
                    model="gpt-4o-mini",
                    messages=[
                        {"role": "system", "content": "You are a helpful assistant that answers questions based on the provided context. Limit your answer to one word."},
                        {"role": "user", "content": f"Context: {repaired_context}\nQuestion: {question}"}
                    ],
                    temperature=0.2,
                    max_tokens=5, # Limit response to one word, but just in case
                )
                answer_repaired = response_repaired.choices[0].message.content.strip()
            except Exception as e:
                print(f"Error generating answer for repaired context: {e}")
                answer_repaired = ""


            # Evaluate answers (F1 and EM already calculated)
            f1_privatized, em_privatized = evaluate_answer(answer_privatized, true_answers)
            f1_repaired, em_repaired = evaluate_answer(answer_repaired, true_answers)

            # Calculate Cosine Similarity

            cosine_privatized = calculate_cosine_similarity(answer_privatized, true_answers, model_sentence)
            cosine_repaired = calculate_cosine_similarity(answer_repaired, true_answers, model_sentence)

            results_list.append({
                "dataframe": df_name,
                "question": question,
                "true_answers": true_answers,
                "answer_privatized": answer_privatized,
                "f1_privatized": f1_privatized,
                "em_privatized": em_privatized,
                "cosine_privatized": cosine_privatized,
                "answer_repaired": answer_repaired,
                "f1_repaired": f1_repaired,
                "em_repaired": em_repaired,
                "cosine_repaired": cosine_repaired,
            })
    # Convert results list to DataFrame for the current round
    df_round_results = pd.DataFrame(results_list)
    all_results.append(df_round_results)

# The list 'all_results' now contains a DataFrame for each round
print(f"\nFinished {num_rounds} rounds of evaluation.")
# You can now proceed to aggregate across the dataframes in all_results

In [ ]:
# --- Combine all Fantasy SQuAD evaluation results and save to CSV ---

# Concatenate all round results into one DataFrame
df_all_results_combined_squad = pd.concat(all_results, ignore_index=True)

# Define output path
csv_output_path_squad = "FantasySQuAD_evaluation_results_inf_rounds.csv"

# Save to CSV
df_all_results_combined_squad.to_csv(csv_output_path_squad, index=False)

print(f"\n✅ All Fantasy SQuAD evaluation results saved to {csv_output_path_squad}")


In [ ]:
# --- Load the Fantasy SQuAD GPT-4 Fill dataset ---
file_path_squad_gpt4_fill = "/content/FantasySQuAD_gpt4_fill_results_with_original.csv"  # adjust if filename differs

try:
    df_squad_gpt4_fill = pd.read_csv(file_path_squad_gpt4_fill)
    print(f"✅ Loaded dataset: {file_path_squad_gpt4_fill}")
    display(df_squad_gpt4_fill.head())
except FileNotFoundError:
    print(f"❌ Error: File not found at {file_path_squad_gpt4_fill}")
    df_squad_gpt4_fill = None


In [ ]:

if df_squad_gpt4_fill is not None:
    results_list = []

    for idx, row in tqdm(df_squad_gpt4_fill.iterrows(), total=len(df_squad_gpt4_fill)):
        question = row.get("question", "")
        true_answers = (
            ast.literal_eval(row["true_answers"])
            if isinstance(row["true_answers"], str)
            else row["true_answers"]
        )

        original_context = row.get("original_context", "")
        privatized_context = row.get("privatized_context", "")
        repaired_context = row.get("repaired_context", "")

        # --- GPT on original ---
        try:
            resp_orig = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": "Answer the question based on the context. Limit your answer to one short span or word."},
                    {"role": "user", "content": f"Context: {original_context}\nQuestion: {question}"}
                ],
                temperature=0.2,
                max_tokens=15,
            )
            ans_orig = resp_orig.choices[0].message.content.strip()
        except Exception as e:
            print(f"⚠️ Error on original context row {idx}: {e}")
            ans_orig = ""

        # --- GPT on privatized (masked) context ---
        try:
            resp_priv = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": "Answer the question based on the context. Limit your answer to one short span or word."},
                    {"role": "user", "content": f"Context: {privatized_context}\nQuestion: {question}"}
                ],
                temperature=0.2,
                max_tokens=15,
            )
            ans_priv = resp_priv.choices[0].message.content.strip()
        except Exception as e:
            print(f"⚠️ Error on privatized context row {idx}: {e}")
            ans_priv = ""

        # --- GPT on repaired (filled) context ---
        try:
            resp_rep = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": "Answer the question based on the context. Limit your answer to one short span or word."},
                    {"role": "user", "content": f"Context: {repaired_context}\nQuestion: {question}"}
                ],
                temperature=0.2,
                max_tokens=15,
            )
            ans_rep = resp_rep.choices[0].message.content.strip()
        except Exception as e:
            print(f"⚠️ Error on repaired context row {idx}: {e}")
            ans_rep = ""

        # --- Evaluate ---
        f1_orig, em_orig = evaluate_answer(ans_orig, true_answers)
        f1_priv, em_priv = evaluate_answer(ans_priv, true_answers)
        f1_rep, em_rep = evaluate_answer(ans_rep, true_answers)

        cos_orig = calculate_cosine_similarity(ans_orig, true_answers, model_sentence)
        cos_priv = calculate_cosine_similarity(ans_priv, true_answers, model_sentence)
        cos_rep = calculate_cosine_similarity(ans_rep, true_answers, model_sentence)

        results_list.append({
            "question": question,
            "true_answers": true_answers,
            "original_context": original_context,
            "privatized_context": privatized_context,
            "repaired_context": repaired_context,
            "answer_original": ans_orig,
            "answer_privatized": ans_priv,
            "answer_repaired": ans_rep,
            "f1_original": f1_orig,
            "em_original": em_orig,
            "cosine_original": cos_orig,
            "f1_privatized": f1_priv,
            "em_privatized": em_priv,
            "cosine_privatized": cos_priv,
            "f1_repaired": f1_rep,
            "em_repaired": em_rep,
            "cosine_repaired": cos_rep,
        })

    # --- Combine & save ---
    df_squad_gpt4_fill_results = pd.DataFrame(results_list)
    csv_output_path = "FantasySQuAD_gpt4_fill_results_with_original.csv"
    df_squad_gpt4_fill_results.to_csv(csv_output_path, index=False)

    print(f"\n✅ Fantasy SQuAD GPT-4 Fill results (with original context) saved to: {csv_output_path}")

    # --- Summary ---
    avg_scores = {
        "mean_f1_original": df_squad_gpt4_fill_results["f1_original"].mean(),
        "mean_em_original": df_squad_gpt4_fill_results["em_original"].mean(),
        "mean_cosine_original": df_squad_gpt4_fill_results["cosine_original"].mean(),
        "mean_f1_privatized": df_squad_gpt4_fill_results["f1_privatized"].mean(),
        "mean_em_privatized": df_squad_gpt4_fill_results["em_privatized"].mean(),
        "mean_cosine_privatized": df_squad_gpt4_fill_results["cosine_privatized"].mean(),
        "mean_f1_repaired": df_squad_gpt4_fill_results["f1_repaired"].mean(),
        "mean_em_repaired": df_squad_gpt4_fill_results["em_repaired"].mean(),
        "mean_cosine_repaired": df_squad_gpt4_fill_results["cosine_repaired"].mean(),
    }

    display(pd.DataFrame([avg_scores]))


In [ ]:
# === Load Files ===
inf_results_path = "/content/FantasySQuAD_evaluation_results_inf_rounds.csv"
baseline_path = "/content/FantasySQuAD_gpt4_fill_results_with_original.csv"

df_inf = pd.read_csv(inf_results_path)
df_base = pd.read_csv(baseline_path)
print(f"✅ Loaded {len(df_inf)} INF results and {len(df_base)} baseline rows.")

# === Extract epsilon/method ===
if "dataframe" in df_inf.columns:
    df_inf["epsilon"] = df_inf["dataframe"].apply(
        lambda x: float(re.search(r"epsilon_([\d.]+)", str(x)).group(1))
        if isinstance(x, str) and "epsilon_" in x else None)
    df_inf["method"] = df_inf["dataframe"].apply(
        lambda x: "Inf Laplace" if "laplace" in str(x).lower()
        else "Inf Polar" if "polar" in str(x).lower()
        else "Unknown")

# === Compute GPT-Judged Correctness (INF results) ===
df_inf["gpt_judged_correct"] = False
for i, row in tqdm(df_inf.iterrows(), total=len(df_inf)):
    question = row.get("question", "")
    pred = row.get("answer_privatized", "")
    try:
        gold = ast.literal_eval(row["true_answers"]) if isinstance(row["true_answers"], str) else row["true_answers"]
    except Exception:
        gold = [str(row.get("true_answers", ""))]
    df_inf.loc[i, "gpt_judged_correct"] = gpt_judge(pred, gold, question)
    time.sleep(0.25)

# === Aggregate Mean Metrics ===
df_inf_clean = df_inf.dropna(subset=["em_privatized", "f1_privatized", "cosine_privatized"])
df_agg = (
    df_inf_clean.groupby(["epsilon", "method"])
    .agg(
        mean_em=("em_privatized", "mean"),
        mean_f1=("f1_privatized", "mean"),
        mean_cosine=("cosine_privatized", "mean"),
        mean_gpt_accuracy=("gpt_judged_correct", "mean"),
    ).reset_index()
)

# === Compute Baseline Metrics ===
baseline_em = df_base.filter(like="em").mean().mean()
baseline_f1 = df_base.filter(like="f1").mean().mean()
baseline_cos = df_base.filter(like="cosine").mean().mean()
print(f"✅ Baselines — EM: {baseline_em:.4f}, F1: {baseline_f1:.4f}, Cosine: {baseline_cos:.4f}")

# === Save Updated File ===
df_inf.to_csv("/content/FantasySQuAD_with_gpt_judged_accuracy.csv", index=False)
print("💾 Saved enhanced FantasySQuAD results with GPT-judged accuracy.")


In [ ]:
# Compute GPT-Judged Correctness (Baseline results)
df_base["gpt_judged_correct"] = False
for i, row in tqdm(df_base.iterrows(), total=len(df_base)):
    question = row.get("question", "")
    pred = row.get("answer_original", "") # Use answer_original for baseline
    try:
        gold = ast.literal_eval(row["true_answers"]) if isinstance(row["true_answers"], str) else row["true_answers"]
    except Exception:
        gold = [str(row.get("true_answers", ""))]
    df_base.loc[i, "gpt_judged_correct"] = gpt_judge(pred, gold, question)
    time.sleep(0.25)

# Calculate and display the mean GPT-judged accuracy for the baseline
baseline_gpt_accuracy = df_base["gpt_judged_correct"].mean()
print(f"✅ Baseline GPT-Judged Accuracy: {baseline_gpt_accuracy:.4f}")

In [ ]:
# Save the updated baseline DataFrame with GPT-judged accuracy
output_baseline_path = "FantasySQuAD_gpt4_fill_results_with_original_and_gpt_accuracy.csv"
df_base.to_csv(output_baseline_path, index=False)
print(f"✅ Updated baseline results (with GPT-judged accuracy) saved to: {output_baseline_path}")

In [ ]:
# === Helper Plot Function ===
def plot_metric(col, ylabel, baseline, title):
    plt.figure(figsize=(12,7))
    for method in df_agg["method"].unique():
        sub = df_agg[df_agg["method"] == method]
        plt.plot(sub["epsilon"], sub[col], marker="o", linestyle="--", label=method)
    if baseline is not None:
        plt.axhline(y=baseline, color="red", linestyle=":", linewidth=2, label="Baseline (GPT-4 Fill)")
    plt.title(f"Fantasy SQuAD — {title} vs Epsilon")
    plt.xlabel("Epsilon")
    plt.ylabel(ylabel)
    plt.ylim(0, 1.05)
    plt.grid(True)
    plt.legend(title="Mechanism")
    plt.tight_layout()
    plt.show()

# === Plot All Metrics ===
plot_metric("mean_em", "Exact Match", baseline_em, "Exact Match (EM)")
plot_metric("mean_f1", "F1 Score", baseline_f1, "F1 Score")
plot_metric("mean_cosine", "Cosine Similarity", baseline_cos, "Cosine Similarity")
plot_metric("mean_gpt_accuracy", "GPT-Judged Accuracy", baseline_gpt_accuracy, "GPT-Judged Accuracy")

In [ ]:

# --- Load Fantasy SQuAD INF results ---
inf_results_path = "/content/FantasySQuAD_evaluation_results_inf_rounds.csv"
df_all_results_combined = pd.read_csv(inf_results_path)
print(f"✅ Loaded {len(df_all_results_combined)} total rows from INF results.")

# --- Extract epsilon values ---
df_all_results_combined['epsilon'] = df_all_results_combined['dataframe'].apply(
    lambda x: float(re.search(r"epsilon_([\d.]+)", str(x)).group(1))
    if isinstance(x, str) and "epsilon_" in x else np.nan
)

# --- Filter and classify methods ---
df_inf_results = df_all_results_combined[
    df_all_results_combined['dataframe'].str.contains('_inf_', case=False, na=False)
].copy()
df_inf_results['method'] = df_inf_results['dataframe'].apply(
    lambda x: 'Inf Laplace' if 'laplace' in str(x).lower() else 'Inf Polar'
)

# --- Aggregate metrics ---
df_aggregated_inf = (
    df_inf_results.groupby(['epsilon', 'method'])
    .agg(
        mean_f1_privatized=('f1_privatized', 'mean'),
        std_f1_privatized=('f1_privatized', 'std'),
        mean_f1_repaired=('f1_repaired', 'mean'),
        std_f1_repaired=('f1_repaired', 'std'),
        mean_cosine_privatized=('cosine_privatized', 'mean'),
        std_cosine_privatized=('cosine_privatized', 'std'),
        mean_cosine_repaired=('cosine_repaired', 'mean'),
        std_cosine_repaired=('cosine_repaired', 'std'),
    )
    .reset_index()
    .sort_values('epsilon')
)
print("✅ Aggregated results ready for plotting.")

# --- Plot: Cosine Similarity (Privatized Context) ---
plt.figure(figsize=(12, 7))

for method, style in [("Inf Laplace", "-"), ("Inf Polar", "--")]:
    df_method = df_aggregated_inf[df_aggregated_inf['method'] == method]
    y = df_method['mean_cosine_privatized']
    yerr = df_method['std_cosine_privatized']

    # Cap error bars within [0, 1]
    upper_error = np.minimum(y + yerr, 1) - y
    lower_error = y - np.maximum(y - yerr, 0)

    plt.errorbar(
        df_method['epsilon'], y,
        yerr=[lower_error, upper_error],
        label=f"{method} (Privatized Context)",
        marker='o', capsize=5, linestyle=style
    )

# --- Add GPT-4 Mask & Fill Baseline if available ---
try:
    df_mask_fill_results = pd.read_csv("/content/FantasySQuAD_gpt4_fill_results_with_original.csv")
    baseline_cosine = df_mask_fill_results['cosine_gpt4_fill'].mean() if 'cosine_gpt4_fill' in df_mask_fill_results.columns \
        else df_mask_fill_results.filter(like="cosine").mean().mean()
    plt.axhline(y=baseline_cosine, color='red', linestyle=':', linewidth=2,
                label=f"GPT-4 Mask & Fill Baseline ({baseline_cosine:.3f})")
except Exception as e:
    print(f"⚠️ Could not load GPT-4 Fill baseline: {e}")

# --- Styling ---
plt.title("Fantasy SQuAD — Cosine Similarity (Privatized Context) vs Epsilon for INF Mechanisms")
plt.xlabel("Epsilon")
plt.ylabel("Average Cosine Similarity")
plt.ylim(0, 1.05)
plt.grid(True)
plt.legend(title="Method", fontsize=10)
plt.tight_layout()
plt.show()

In [ ]:
# Extract tau values from the 'dataframe' column (if not already done in df_all_results_combined)
if 'tau' not in df_all_results_combined.columns:
    df_all_results_combined['tau'] = df_all_results_combined['dataframe'].apply(lambda x: float(x.split('_')[-1]))

# Create the plot for F1 scores with shaded standard deviation
plt.figure(figsize=(10, 6))
sns.lineplot(data=df_all_results_combined, x='tau', y='f1_privatized', label='Privatized Context', marker='o', errorbar='sd')
sns.lineplot(data=df_all_results_combined, x='tau', y='f1_repaired', label='Repaired Context', marker='o', errorbar='sd')

plt.title('Average F1 Score vs. Importance Threshold')
plt.xlabel('Importance Threshold')
plt.ylabel('Average F1 Score')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
# Extract tau values from the 'dataframe' column (if not already done in df_all_results_combined)
if 'tau' not in df_all_results_combined.columns:
    df_all_results_combined['tau'] = df_all_results_combined['dataframe'].apply(lambda x: float(x.split('_')[-1]))

# Create the plot for Cosine Similarity with shaded standard deviation
plt.figure(figsize=(10, 6))
sns.lineplot(data=df_all_results_combined, x='tau', y='cosine_privatized', label='Privatized Context', marker='o', errorbar='sd')
sns.lineplot(data=df_all_results_combined, x='tau', y='cosine_repaired', label='Repaired Context', marker='o', errorbar='sd')

plt.title('Average Cosine Similarity vs. Importance Threshold')
plt.xlabel('Importance Threshold')
plt.ylabel('Average Cosine Similarity Score')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
# Extract tau values from the 'dataframe' column (if not already done in df_all_results_combined)
if 'tau' not in df_all_results_combined.columns:
    df_all_results_combined['tau'] = df_all_results_combined['dataframe'].apply(lambda x: float(x.split('_')[-1]))

# Create the plot for F1 scores with confidence intervals
plt.figure(figsize=(10, 6))
sns.lineplot(data=df_all_results_combined, x='tau', y='f1_privatized', label='Privatized Context', marker='o', errorbar=('ci', 95))
sns.lineplot(data=df_all_results_combined, x='tau', y='f1_repaired', label='Repaired Context', marker='o', errorbar=('ci', 95))

plt.title('Average F1 Score vs. Importance Threshold (95% Confidence Interval)')
plt.xlabel('Importance Threshold')
plt.ylabel('Average F1 Score')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
# Extract tau values from the 'dataframe' column (if not already done in df_all_results_combined)
if 'tau' not in df_all_results_combined.columns:
    df_all_results_combined['tau'] = df_all_results_combined['dataframe'].apply(lambda x: float(x.split('_')[-1]))

# Create the plot for Cosine Similarity with confidence intervals
plt.figure(figsize=(10, 6))
sns.lineplot(data=df_all_results_combined, x='tau', y='cosine_privatized', label='Privatized Context', marker='o', errorbar=('ci', 95))
sns.lineplot(data=df_all_results_combined, x='tau', y='cosine_repaired', label='Repaired Context', marker='o', errorbar=('ci', 95))

plt.title('Average Cosine Similarity vs. Importance Threshold (95% Confidence Interval)')
plt.xlabel('Importance Threshold')
plt.ylabel('Average Cosine Similarity Score')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
df_mask_fill = pd.read_csv('/content/squad_gpt4_mask_fill.csv')
display(df_mask_fill.head())

In [ ]:
results_list_mask_fill = []

for index, row in df_mask_fill.iterrows():
    question = row["question"]
    # Safely evaluate the string representation of the list of answers
    try:
        true_answers = eval(row["answers"])
        if not isinstance(true_answers, list):
             true_answers = [true_answers] # Ensure it's a list even if eval returns a single item
    except Exception as e:
        print(f"Error evaluating answers for row {index}: {e}")
        true_answers = []

    privatized_context = row["privatized_context"]
    repaired_context = row["repaired_context"]

    # Generate answer for privatized context
    try:
        response_privatized = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that answers questions based on the provided context. Limit your answer to one word."},
                {"role": "user", "content": f"Context: {privatized_context}\nQuestion: {question}"}
            ],
            temperature=0.2,
            max_tokens=5,
        )
        answer_privatized = response_privatized.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error generating answer for privatized context in row {index}: {e}")
        answer_privatized = ""

    # Generate answer for repaired context
    try:
        response_repaired = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that answers questions based on the provided context. Limit your answer to one word."},
                {"role": "user", "content": f"Context: {repaired_context}\nQuestion: {question}"}
            ],
            temperature=0.2,
            max_tokens=5,
        )
        answer_repaired = response_repaired.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error generating answer for repaired context in row {index}: {e}")
        answer_repaired = ""

    # Evaluate answers
    f1_privatized, em_privatized = evaluate_answer(answer_privatized, true_answers)
    f1_repaired, em_repaired = evaluate_answer(answer_repaired, true_answers)

    # Calculate Cosine Similarity
    cosine_privatized = calculate_cosine_similarity(answer_privatized, true_answers, model_sentence)
    cosine_repaired = calculate_cosine_similarity(answer_repaired, true_answers, model_sentence)

    results_list_mask_fill.append({
        "question": question,
        "true_answers": true_answers,
        "answer_privatized": answer_privatized,
        "f1_privatized": f1_privatized,
        "em_privatized": em_privatized,
        "cosine_privatized": cosine_privatized,
        "answer_repaired": answer_repaired,
        "f1_repaired": f1_repaired,
        "em_repaired": em_repaired,
        "cosine_repaired": cosine_repaired,
    })

# Convert results list to DataFrame
df_mask_fill_results = pd.DataFrame(results_list_mask_fill)
display(df_mask_fill_results.head())

In [ ]:
average_scores = {
    'mean_f1_privatized': df_mask_fill_results['f1_privatized'].mean(),
    'mean_em_privatized': df_mask_fill_results['em_privatized'].mean(),
    'mean_cosine_privatized': df_mask_fill_results['cosine_privatized'].mean(),
    'mean_f1_repaired': df_mask_fill_results['f1_repaired'].mean(),
    'mean_em_repaired': df_mask_fill_results['em_repaired'].mean(),
    'mean_cosine_repaired': df_mask_fill_results['cosine_repaired'].mean()
}

# Convert to DataFrame for display
df_average_scores = pd.DataFrame([average_scores])
display(df_average_scores)

In [ ]:
# Save the results DataFrame to a CSV file
csv_output_path_mask_fill = "squad_gpt4_mask_fill_results.csv"
df_mask_fill_results.to_csv(csv_output_path_mask_fill, index=False)

print(f"Evaluation results for mask fill saved to {csv_output_path_mask_fill}")

In [ ]:
# --- load all UNIFORM_Polar and UNIFORM_Laplace datasets ---
csv_files_uniform_polar = glob.glob("/content/squad_uniform_polar_sweep_avg_epsilon_*.csv")
csv_files_uniform_laplace = glob.glob("/content/squad_uniform_laplace_sweep_avg_epsilon_*.csv")

csv_files_uniform = csv_files_uniform_polar + csv_files_uniform_laplace
print("Files found by glob:", csv_files_uniform)

dataframes_uniform = {}

for file in csv_files_uniform:
    df_name = file.replace(".csv", "")
    dataframes_uniform[df_name] = pd.read_csv(file)

print("Number of UNIFORM dataframes loaded:", len(dataframes_uniform))

In [ ]:
all_results_uniform = []
num_rounds = 3

for round_num in range(num_rounds):
    print(f"--- Starting Round {round_num + 1}/{num_rounds} (UNIFORM) ---")
    results_list = []

    for df_name, df in dataframes_uniform.items():
        print(f"Processing DataFrame: {df_name}")
        for index, row in df.iterrows():
            question = row["question"]
            try:
                true_answers = eval(row["answers"])
                if not isinstance(true_answers, list):
                     true_answers = [true_answers]
            except Exception as e:
                print(f"Error evaluating answers for row {index} in {df_name}: {e}")
                true_answers = []

            privatized_context = row["privatized_context"]
            repaired_context = row["repaired_context"]

            # Generate answer for privatized context
            try:
                response_privatized = client.chat.completions.create(
                    model="gpt-4o-mini",
                    messages=[
                        {"role": "system", "content": "You are a helpful assistant that answers questions based on the provided context. Limit your answer to one word."},
                        {"role": "user", "content": f"Context: {privatized_context}\nQuestion: {question}"}
                    ],
                    temperature=0.2,
                    max_tokens=5,
                )
                answer_privatized = response_privatized.choices[0].message.content.strip()
            except Exception as e:
                print(f"Error generating answer for privatized context in row {index} of {df_name}: {e}")
                answer_privatized = ""

            # Generate answer for repaired context
            try:
                response_repaired = client.chat.completions.create(
                    model="gpt-4o-mini",
                    messages=[
                        {"role": "system", "content": "You are a helpful assistant that answers questions based on the provided context. Limit your answer to one word."},
                        {"role": "user", "content": f"Context: {repaired_context}\nQuestion: {question}"}
                    ],
                    temperature=0.2,
                    max_tokens=5,
                )
                answer_repaired = response_repaired.choices[0].message.content.strip()
            except Exception as e:
                print(f"Error generating answer for repaired context in row {index} of {df_name}: {e}")
                answer_repaired = ""

            # Evaluate answers
            f1_privatized, em_privatized = evaluate_answer(answer_privatized, true_answers)
            f1_repaired, em_repaired = evaluate_answer(answer_repaired, true_answers)

            # Calculate Cosine Similarity
            cosine_privatized = calculate_cosine_similarity(answer_privatized, true_answers, model_sentence)
            cosine_repaired = calculate_cosine_similarity(answer_repaired, true_answers, model_sentence)

            results_list.append({
                "dataframe": df_name,
                "question": question,
                "true_answers": true_answers,
                "answer_privatized": answer_privatized,
                "f1_privatized": f1_privatized,
                "em_privatized": em_privatized,
                "cosine_privatized": cosine_privatized,
                "answer_repaired": answer_repaired,
                "f1_repaired": f1_repaired,
                "em_repaired": em_repaired,
                "cosine_repaired": cosine_repaired,
            })
    df_round_results = pd.DataFrame(results_list)
    all_results_uniform.append(df_round_results)

print(f"\nFinished {num_rounds} rounds of evaluation for UNIFORM files.")

df_all_results_combined_uniform = pd.concat(all_results_uniform, ignore_index=True)

csv_output_path_uniform = "evaluation_results_uniform_rounds.csv"
df_all_results_combined_uniform.to_csv(csv_output_path_uniform, index=False)

print(f"All evaluation results for UNIFORM files saved to {csv_output_path_uniform}")

In [ ]:
# Extract epsilon values from the 'dataframe' column
df_all_results_combined_uniform['epsilon'] = df_all_results_combined_uniform['dataframe'].apply(lambda x: float(x.split('_')[-1]))

# Identify method type for uniform dataframes
df_uniform_results = df_all_results_combined_uniform.copy()

df_uniform_results['method'] = df_uniform_results['dataframe'].apply(
    lambda x: 'Uniform Laplace' if 'laplace' in x.lower() else 'Uniform Polar'
)

# Aggregate results by epsilon and method
df_aggregated_uniform = df_uniform_results.groupby(['epsilon', 'method']).agg(
    mean_f1_privatized=('f1_privatized', 'mean'),
    std_f1_privatized=('f1_privatized', 'std'),
    mean_f1_repaired=('f1_repaired', 'mean'),
    std_f1_repaired=('f1_repaired', 'std'),
    mean_cosine_privatized=('cosine_privatized', 'mean'),
    std_cosine_privatized=('cosine_privatized', 'std'),
    mean_cosine_repaired=('cosine_repaired', 'mean'),
    std_cosine_repaired=('cosine_repaired', 'std')
).reset_index()

# Sort by epsilon to ensure correct line plotting
df_aggregated_uniform = df_aggregated_uniform.sort_values('epsilon')

# --- Plot Average Cosine Similarity (Privatized Context) vs. Epsilon for Uniform Methods with Capped Standard Deviation ---
plt.figure(figsize=(12, 7))

# Plot for Uniform Laplace
df_uniform_laplace = df_aggregated_uniform[df_aggregated_uniform['method'] == 'Uniform Laplace']
y_uniform_laplace = df_uniform_laplace['mean_cosine_privatized']
yerr_uniform_laplace = df_uniform_laplace['std_cosine_privatized']
# Cap the upper error bar at 1
upper_error_uniform_laplace = np.minimum(y_uniform_laplace + yerr_uniform_laplace, 1) - y_uniform_laplace
lower_error_uniform_laplace = y_uniform_laplace - (y_uniform_laplace - yerr_uniform_laplace)

plt.errorbar(
    df_uniform_laplace['epsilon'],
    y_uniform_laplace,
    yerr=[lower_error_uniform_laplace, upper_error_uniform_laplace],
    label='Uniform Laplace (Privatized Context)',
    marker='o',
    capsize=5,
    linestyle='-'
)

# Plot for Uniform Polar
df_uniform_polar = df_aggregated_uniform[df_aggregated_uniform['method'] == 'Uniform Polar']
y_uniform_polar = df_uniform_polar['mean_cosine_privatized']
yerr_uniform_polar = df_uniform_polar['std_cosine_privatized']
# Cap the upper error bar at 1
upper_error_uniform_polar = np.minimum(y_uniform_polar + yerr_uniform_polar, 1) - y_uniform_polar
lower_error_uniform_polar = y_uniform_polar - (y_uniform_polar - yerr_uniform_polar)

plt.errorbar(
    df_uniform_polar['epsilon'],
    y_uniform_polar,
    yerr=[lower_error_uniform_polar, upper_error_uniform_polar],
    label='Uniform Polar (Privatized Context)',
    marker='o',
    capsize=5,
    linestyle='--'
)

# Add horizontal line for GPT4 Mask Fill (Privatized Context)
# Assuming df_mask_fill_results DataFrame is available from previous steps and average_scores was calculated
if 'average_scores' in globals() and 'mean_cosine_privatized' in average_scores:
    mean_cosine_mask_fill_privatized = average_scores['mean_cosine_privatized']
    plt.axhline(y=mean_cosine_mask_fill_privatized, color='red', linestyle=':', label='GPT4 Mask Fill (Privatized Context)')
else:
    print("Warning: Could not find mask fill average cosine similarity for privatized context.")


plt.title('Average Cosine Similarity (Privatized Context) vs. Epsilon for Uniform Methods (Capped Standard Deviation)')
plt.xlabel('Epsilon')
plt.ylabel('Average Cosine Similarity Score')
plt.grid(True)
plt.legend(title='Method and Context')
plt.show()

In [ ]:
# Load the specific dataset
file_path = "/content/squad_stamp_Laplace_sweep_avg_epsilon_1868.98.csv"
try:
    df_specific = pd.read_csv(file_path)
    print(f"Loaded dataset: {file_path}")
except FileNotFoundError:
    print(f"Error: File not found at {file_path}")
    df_specific = None

if df_specific is not None:
    results_list_specific = []

    for index, row in df_specific.iterrows():
        question = row["question"]
        try:
            true_answers = eval(row["answers"])
            if not isinstance(true_answers, list):
                 true_answers = [true_answers]
        except Exception as e:
            print(f"Error evaluating answers for row {index}: {e}")
            true_answers = []

        privatized_context = row["privatized_context"]
        repaired_context = row["repaired_context"]

        # Generate answer for privatized context
        try:
            response_privatized = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant that answers questions based on the provided context. Limit your answer to one word."},
                    {"role": "user", "content": f"Context: {privatized_context}\nQuestion: {question}"}
                ],
                temperature=0.2,
                max_tokens=5, # Limit response to one word, but just in case
            )
            predicted_answer_privatized = response_privatized.choices[0].message.content.strip()
        except Exception as e:
            print(f"Error generating answer for privatized context: {e}")
            predicted_answer_privatized = ""


        # Generate answer for repaired context
        try:
            response_repaired = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant that answers questions based on the provided context. Limit your answer to one word."},
                    {"role": "user", "content": f"Context: {repaired_context}\nQuestion: {question}"}
                ],
                temperature=0.2,
                max_tokens=5, # Limit response to one word, but just in case
            )
            predicted_answer_repaired = response_repaired.choices[0].message.content.strip()
        except Exception as e:
            print(f"Error generating answer for repaired context: {e}")
            predicted_answer_repaired = ""


        # Calculate Cosine Similarity using the pre-trained model_sentence
        cosine_privatized = calculate_cosine_similarity(predicted_answer_privatized, true_answers, model_sentence)
        cosine_repaired = calculate_cosine_similarity(predicted_answer_repaired, true_answers, model_sentence)

        results_list_specific.append({
            "question": question,
            "true_answers": true_answers,
            "answer_privatized": predicted_answer_privatized,
            "cosine_privatized": cosine_privatized,
            "answer_repaired": predicted_answer_repaired,
            "cosine_repaired": cosine_repaired,
        })

    df_specific_results = pd.DataFrame(results_list_specific)

    # Calculate average cosine similarity
    average_cosine_privatized = df_specific_results['cosine_privatized'].mean()
    average_cosine_repaired = df_specific_results['cosine_repaired'].mean()

    print(f"\nAverage Cosine Similarity for {file_path}:")
    print(f"  Privatized Context: {average_cosine_privatized:.4f}")
    print(f"  Repaired Context: {average_cosine_repaired:.4f}")